In [1]:
# imports(default)
import sys
from glob import glob
import numpy as np
import tensorflow as tf
import gensim
import MeCab
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import *
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.optimizers import *
keras = tf.keras
sys.path.append("../")
from models import *
from utils import *

/home/shido/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
char_files = glob("../dataset/charactor/*.txt")
conv_files = glob("../dataset/conversation/*.txt")

In [3]:
char_texts = [read_file(file) for file in char_files]
conv_texts = [read_file(file) for file in conv_files]

In [4]:
parser = Parser()

In [5]:
vocab = set()
for conv in conv_texts:
    for line in conv:
        vocab = vocab.union(set(parser.parse(line)))
for conv in char_texts:
    for line in conv:
        vocab = vocab.union(set(parser.parse(line)))

In [6]:
word2id = {}
id2word = {}

for e, word in enumerate(["<PAD>", "<BOS>", "<EOS>", "<UNK>"] + sorted(list(vocab))):
    word2id[word] = e
    id2word[e] = word

In [7]:
tokenizer = Tokenizer(word2id, id2word)
char_seqs = [[tokenizer.encode(parser.parse(line)) for line in text] for text in char_texts]
conv_seqs = [[tokenizer.encode(parser.parse(line)) for line in text] for text in conv_texts]

In [8]:
EMBEDDING_SIZE = 200
NUM_UNITS = 400
SEQ_LEN = 150
BEAM_WIDTH = 3
BATCH_SIZE = 256
VOCAB = len(word2id)

In [46]:
class Classifier:
    def __init__(self, EMBEDDING_SIZE, NUM_UNITS, VOCAB):
        
        masking_func = lambda inputs, mask: K.repeat_elements(K.expand_dims(mask, -1), VOCAB, 1)
        self.l_one_hot = Lambda(lambda x: K.one_hot(x, VOCAB), mask=masking_func)
        self.l_embed = Dense(EMBEDDING_SIZE, use_bias=False)
        self.l_lstm = LSTM(NUM_UNITS, dropout=.2, recurrent_dropout=.2, return_sequences=True)
        self.l_dense = Dense(1, activation='sigmoid', name="y")
        
        inputs = Input([None], dtype="int32", name="x1")
        y = Masking(0)(inputs)
        y = self.l_one_hot(y)
        print(y)
        y = self.l_embed(y)
        print(y)
        y = self.l_lstm(y)
        print(y)
#         y = self.l_dense(y)

        self.model = Model(inputs=[inputs], outputs=y)

In [47]:
clf_conv = []
for text in conv_seqs:
    clf_conv += text[1::2]
clf_char = []
for text in char_seqs:
    clf_char += text
clf_x = pad_sequences(clf_char + clf_conv, SEQ_LEN, padding="post", truncating="post")
clf_y = [1 for _ in clf_char] + [0 for _ in clf_conv]
perm = np.random.permutation(len(clf_x))
clf_x = clf_x[perm]
clf_y = [clf_y[i] for i in perm]

In [48]:
clf = Classifier(EMBEDDING_SIZE, NUM_UNITS, VOCAB)
clf.model.compile(Adam(1e-3), loss="binary_crossentropy", metrics=["accuracy"])

Tensor("lambda_9/one_hot:0", shape=(?, ?, 5753), dtype=float32)
Tensor("dense_7/Tensordot:0", shape=(?, ?, 200), dtype=float32)
Tensor("lstm_6/transpose_2:0", shape=(?, ?, 400), dtype=float32)


In [15]:
clf.model.fit(clf_x, clf_y, validation_split=.2, shuffle=True, batch_size=64, epochs=5)

ValueError: Error when checking target: expected lstm_1 to have 3 dimensions, but got array with shape (3942, 1)

In [49]:
clf.model.predict(clf_x[0:10])[0].shap

InvalidArgumentError: Max scatter index must be < array size (5752 vs. 150)
	 [[Node: lstm_6/TensorArrayUnstack_1/TensorArrayScatter/TensorArrayScatterV3 = TensorArrayScatterV3[T=DT_BOOL, _device="/job:localhost/replica:0/task:0/device:CPU:0"](lstm_6/TensorArray_2, lstm_6/TensorArrayUnstack_1/range/_365, lstm_6/transpose_1, lstm_6/TensorArray_2:1)]]

Caused by op 'lstm_6/TensorArrayUnstack_1/TensorArrayScatter/TensorArrayScatterV3', defined at:
  File "/home/shido/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/shido/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/shido/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/shido/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch

In [18]:
clf.model.save("../models/clf.keras")

In [72]:
query = ["前川ですが", "みくだにゃ"]
query = [tokenizer.encode(parser.parse(q)) for q in query]
query = pad_sequences(query, 150)

clf.model.predict(query)

array([[0.22352591],
       [0.2276501 ]], dtype=float32)